In [52]:
import pdb
import requests
from bs4 import BeautifulSoup   

In [53]:
response = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')

In [54]:
print(response)

<Response [200]>


In [55]:
response.status_code

200

Адрес, по которому мы обращались, возвращает результат в json формате. Эти данные уже лежат в атрибуте text в полученном ответе response:

In [56]:
print(response.text)

{
    "Date": "2021-05-08T11:30:00+03:00",
    "PreviousDate": "2021-05-07T11:30:00+03:00",
    "PreviousURL": "\/\/www.cbr-xml-daily.ru\/archive\/2021\/05\/07\/daily_json.js",
    "Timestamp": "2021-05-09T23:00:00+03:00",
    "Valute": {
        "AUD": {
            "ID": "R01010",
            "NumCode": "036",
            "CharCode": "AUD",
            "Nominal": 1,
            "Name": "Австралийский доллар",
            "Value": 57.6418,
            "Previous": 57.7524
        },
        "AZN": {
            "ID": "R01020A",
            "NumCode": "944",
            "CharCode": "AZN",
            "Nominal": 1,
            "Name": "Азербайджанский манат",
            "Value": 43.6358,
            "Previous": 43.8946
        },
        "GBP": {
            "ID": "R01035",
            "NumCode": "826",
            "CharCode": "GBP",
            "Nominal": 1,
            "Name": "Фунт стерлингов Соединенного королевства",
            "Value": 103.2288,
            "Previous": 103.6695
 

Сейчас текст хранится просто в строковой переменной. Далее мы можем превратить эту строку в словарь. Сделать это можно с помощью JSON-парсера python, либо воспользовавшись методом json, который уже встроен в объект ответа response:

In [57]:
currences = response.json()

In [58]:
currences

{'Date': '2021-05-08T11:30:00+03:00',
 'PreviousDate': '2021-05-07T11:30:00+03:00',
 'PreviousURL': '//www.cbr-xml-daily.ru/archive/2021/05/07/daily_json.js',
 'Timestamp': '2021-05-09T23:00:00+03:00',
 'Valute': {'AUD': {'ID': 'R01010',
   'NumCode': '036',
   'CharCode': 'AUD',
   'Nominal': 1,
   'Name': 'Австралийский доллар',
   'Value': 57.6418,
   'Previous': 57.7524},
  'AZN': {'ID': 'R01020A',
   'NumCode': '944',
   'CharCode': 'AZN',
   'Nominal': 1,
   'Name': 'Азербайджанский манат',
   'Value': 43.6358,
   'Previous': 43.8946},
  'GBP': {'ID': 'R01035',
   'NumCode': '826',
   'CharCode': 'GBP',
   'Nominal': 1,
   'Name': 'Фунт стерлингов Соединенного королевства',
   'Value': 103.2288,
   'Previous': 103.6695},
  'AMD': {'ID': 'R01060',
   'NumCode': '051',
   'CharCode': 'AMD',
   'Nominal': 100,
   'Name': 'Армянских драмов',
   'Value': 14.2175,
   'Previous': 14.3175},
  'BYN': {'ID': 'R01090B',
   'NumCode': '933',
   'CharCode': 'BYN',
   'Nominal': 1,
   'Name': 

In [59]:
currences['Valute']['CZK']['Name']

'Чешских крон'

Оформляем функцию
В завершение давайте оформим наши вычисления в отдельную функцию, которой будет удобно пользоваться. На вход она должна принимать два параметра:

1. Название валюты currency. Например, 'EUR' или 'USD'.

2. Формат ответа format. При значении 'full' будем отдавать все, что знаем о валюте. Например, для currency = 'USD':

In [60]:
def exchange_rates(currency, format='full'):    
    url = 'https://www.cbr-xml-daily.ru/daily_json.js'  
    response = requests.get(url).json()['Valute']    
    data = response[currency]     
    if format == 'full':    
        return data      
    elif format == 'value':    
        return data['Value']    

Напишите функцию currency_name, которая по ID валюты возвращает ее название на русском языке.

In [61]:
def currency_name(id_):
    url = 'https://www.cbr-xml-daily.ru/daily_json.js'  
    response = requests.get(url).json()['Valute']
#     pdb.set_trace()
    for valute in response:
        if response[valute]['ID'] == id_:
            return response[valute]['Name']
        
    


In [62]:
currency_name('R01700J')

'Турецких лир'

# HTML-страницы

Довольно часто приходится добывать информацию не из удобно форматированного json-файла, а прямо с HTML-страниц. Получить содержимое страницы в большинстве случаев несложно, труднее извлечь из HTML-кода нужную информацию. В качестве примера мы рассмотрим страницу новости, из которой будем доставать полезную информацию: 1) заголовок страницы; 2) дату публикации; 3) текст публикации; 4) ссылки на странице.

In [63]:
url = 'https://nplus1.ru/news/2019/06/04/slothbot'   
  
response = requests.get(url)  
print(response.status_code)
print(response.text)

200
<!doctype html>
<html class="no-js bg-fixed _no-bg" style="background-image:url(https://nplus1.ru/images/2019/06/04/b32b62189fb87cce895e229e1d6d27b4.jpeg)" lang="">
<head>
    
    <meta charset="utf-8">
    <meta http-equiv="x-ua-compatible" content="ie=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">
        <meta name="apple-mobile-web-app-capable" content="yes">
    <meta name="apple-mobile-web-app-status-bar-style" content="black">
                <meta name='wmail-verification' content='7991d7eb02d759f05b9050e111a7e3eb' />

    <link rel="canonical" href="https://nplus1.ru/news/2019/06/04/slothbot" />

        <title>Робота-ленивца научили лазать по паутине из тросов</title>

    	    <meta itemprop="datePublished" content="2019-06-04"/>
	
	    <meta name="mediator_author" content="Григорий Копиев"/> 
	
        <!-- amp page -->
    <link rel="amphtml" href="https://nplus1.ru/news/2019/06/04/slothbot/amp">
    

        <!-- for Google -->
     

# Библиотека BeautifulSoup

In [64]:
url = 'https://nplus1.ru/news/2019/06/04/slothbot'   
response = requests.get(url)  

page = BeautifulSoup(response.text, 'html.parser')

In [65]:
page.title.text

'Робота-ленивца научили лазать по паутине из тросов'

# Данные со страницы

Напишите функцию wiki_header, которая по адресу страницы возвращает заголовок для статей на википедии

In [66]:
def wiki_header(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup.find('h1').text

In [67]:
wiki_header('https://en.wikipedia.org/wiki/Operating_system')

'Operating system'

У вас есть переменная page, в которой хранится содержимое html-страницы. На странице есть элемент в тэге span, у которого атрибут id равен 'target'. Напишите строчку кода, которая присвоит текст этого элемента переменной value.

Запишите ваш код в одну строку без пробелов. Используйте апострофы для передачи параметров, содержащих значение тэга и его id.

In [68]:
# value=page.find('span',class='target')

# Сбор нескольких элементов

Рассмотрим случай, когда нам нужно сразу много элементов. Пусть мы хотим получить название всех ссылок на странице в википедии про языки программирования.

Для ссылок существует тэг <a></a>.  Давайте попробуем использовать find  

In [69]:
url = 'https://en.wikipedia.org/wiki/List_of_programming_languages'  
  
response = requests.get(url)  
page = BeautifulSoup(response.text, 'html.parser')  
page.find('a')  

<a id="top"></a>

Метод find возвращает только первый подходящий элемент. Если нам надо их больше, нужно воспользоваться методом find_all



In [70]:
links = page.find_all('a')  
# Посмотрим, сколько всего мы получили  
print(len(links))

927


In [71]:
# Посмотрим на некоторые из ссылок  
print([link.text for link in links[500:510]])  

['Orwell', 'Oxygene', 'Oz', 'edit', 'P', 'P4', 'P′′', 'ParaSail (programming language)', 'PARI/GP', 'Pascal']


Ещё одна полезная вещь:  последовательный поиск, т.е. мы можем найти сначала один элемент, а потом сделать внутри него второй поиск. Давайте выведем названия всех ссылок для языков программирования, которые начинаются на литеру "A".

In [72]:
# Получаем все элементы с тегом 'div' и классом 'div-col'  
all_blocks = page.find_all('div', class_='div-col')  
  
# Выбираем первый по счету блок  
first_block = all_blocks[0]  
# Берём оттуда ссылки (ограничимся первыми десятью)
links = first_block.find_all('a')
print([link.text for link in links[:10]])

['A.NET (A#/A sharp)', 'A-0 System', 'A+ (A (plus)', 'ABAP', 'ABC', 'ABC ALGOL', 'ACC', 'Accent (Rational Synergy)', 'Ace DASL (Distributed Application Specification Language)', 'Action!']


# Напишите функцию get_actors, которая по ссылке на страницу фильма на кинопоиске возвращает список актёров из колонки справа.



In [73]:
def get_actors(url):
    response = requests.get(url)
    page = BeautifulSoup(response.text, 'html.parser')
    blocks = page.find_all('ul', class_='styles_list__I97eu')
#     actors = blocks.find('li')
    return blocks

In [74]:
get_actors('https://www.kinopoisk.ru/film/42326/')

[<ul class="styles_list__I97eu"><li class="styles_root__-coRa styles_rootInLight__33a_D" data-tid="615a7f9d"><a class="styles_link__1dkjp" href="/name/102198/" itemprop="actor">Мария Виноградова</a></li><li class="styles_root__-coRa styles_rootInLight__33a_D" data-tid="615a7f9d"><a class="styles_link__1dkjp" href="/name/122798/" itemprop="actor">Олег Табаков</a></li><li class="styles_root__-coRa styles_rootInLight__33a_D" data-tid="615a7f9d"><a class="styles_link__1dkjp" href="/name/261607/" itemprop="actor">Лев Дуров</a></li><li class="styles_root__-coRa styles_rootInLight__33a_D" data-tid="615a7f9d"><a class="styles_link__1dkjp" href="/name/273117/" itemprop="actor">Борис Новиков</a></li><li class="styles_root__-coRa styles_rootInLight__33a_D" data-tid="615a7f9d"><a class="styles_link__1dkjp" href="/name/277168/" itemprop="actor">Герман Качин</a></li><li class="styles_root__-coRa styles_rootInLight__33a_D" data-tid="615a7f9d"><a class="styles_link__1dkjp" href="/name/245645/" itempro

# HTML-таблицы

Метод read_html из pandas умеет автоматически находить на HTML-странице таблицы и возвращать их списком из датафреймов:

In [75]:
import pandas as pd  
url = 'https://www.cbr.ru/key-indicators/'  
pd.read_html(url)[0]

,0,1,2
0,валюта,07.05.2021,08.05.2021
1,Доллар США USD,745770,741373
2,Евро EUR,896788,895060


Притворяемся человеком

In [76]:
from bs4 import BeautifulSoup  
import pandas as pd  
import requests  
  
url = 'https://www.cbr.ru/key-indicators/'
soup = BeautifulSoup(requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).text, 'html.parser')  

In [77]:
all_blocks = soup.find_all('div', class_='key-indicator_content offset-md-2')
data = all_blocks[1].find('table') 

In [78]:
df = pd.read_html(str(data))[0]

In [79]:
df

,0,1,2
0,рублей за грамм,07.05.2021,08.05.2021
1,Золото Au,"4 299,44","4 339,29"
2,Серебро Ag,6408,6525
3,Платина Pt,"2 961,16","2 979,46"
4,Палладий Pd,"7 061,24","6 981,47"


Напишите программу, которая забирает данные из таблицы рейтинга банков с https://www.banki.ru/banks/ratings/, делает из него датафрейм и сохраняет его в переменую df.

In [200]:
url = 'https://www.banki.ru/banks/ratings/'
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
page = BeautifulSoup(response.text, 'html.parser')

blocks = page.find_all('div', class_='layout-column-full')
table = blocks[2].find('table', attrs={'data-test': 'rating-table',
              'class': 'standard-table standard-table--row-highlight margin-bottom-small margin-top-x-small', }, )

df = pd.read_html(str(table))[0]

In [202]:
df

место                  название банка лицензия №, Регион  \
    место                  название банка лицензия №, Регион   
0       1            СберБанк лицензия № 1481, Москва и обл.   
1       2        ВТБ лицензия № 1000, Санкт-Петербург и обл.   
2       3          Газпромбанк лицензия № 354, Москва и обл.   
3       4  Национальный Клиринговый Центр лицензия № 3466...   
4       5          Альфа-Банк лицензия № 1326, Москва и обл.   
5       6      Россельхозбанк лицензия № 3349, Москва и обл.   
6       7  Московский Кредитный Банк лицензия № 1978, Мос...   
7       8       Банк Открытие лицензия № 2209, Москва и обл.   
8       9        Совкомбанк лицензия № 963, Костромская обл.   
9      10      Райффайзенбанк лицензия № 3292, Москва и обл.   
10     11             Росбанк лицензия № 2272, Москва и обл.   
11     12         ЮниКредит Банк лицензия № 1, Москва и обл.   
12     13               Траст лицензия № 3279, Москва и обл.   
13     14      Россия лицензия № 328, Санкт-Петербург и обл.   
14     15  Всероссийский Банк Развития Регионов лицензия ...   
15     16       Тинькофф Банк лицензия № 2673, Москва и обл.   
16     17  Санкт-Петербург лицензия № 436, Санкт-Петербур...   
17     18            Ситибанк лицензия № 2557, Москва и обл.   
18  19 +2         Банк ДОМ.РФ лицензия № 2312, Москва и обл.   
19     20         Новикомбанк лицензия № 2546, Москва и обл.   
20  21 +1            СМП Банк лицензия № 3368, Москва и обл.   
21  22 −3                 Ак Барс лицензия № 2590, Татарстан   
22     23          Банк Уралсиб лицензия № 30, Москва и обл.   
23     24             БМ-Банк лицензия № 2748, Москва и обл.   
24     25           Почта Банк лицензия № 650, Москва и обл.   
25     26  Московский Областной Банк лицензия № 1751, Мос...   
26  27 +1            Пересвет лицензия № 2110, Москва и обл.   
27  28 +1  Московский Индустриальный Банк лицензия № 912,...   
28  29 +1    Русский Стандарт лицензия № 2289, Москва и обл.   
29  30 −3  Национальный Расчетный Депозитарий лицензия № ...   
30  31 +1                  РНКБ лицензия № 1354, Симферополь   
31  32 −1  Уральский Банк Реконструкции и Развития лиценз...   
32     33        Абсолют Банк лицензия № 2306, Москва и обл.   
33     34     Хоум Кредит Банк лицензия № 316, Москва и обл.   
34     35          Банк Зенит лицензия № 3255, Москва и обл.   
35     36      Восточный Банк лицензия № 1460, Амурская обл.   
36     37            МТС Банк лицензия № 2268, Москва и обл.   
37     38    Транскапиталбанк лицензия № 2210, Москва и обл.   
38     39        Росэксимбанк лицензия № 2790, Москва и обл.   
39     40      Инвестторгбанк лицензия № 2763, Москва и обл.   
40     41            ИНГ Банк лицензия № 2495, Москва и обл.   
41     42           Локо-Банк лицензия № 2707, Москва и обл.   
42  43 +2        Сетелем Банк лицензия № 2168, Москва и обл.   
43  44 −1    Ренессанс Кредит лицензия № 3354, Москва и обл.   
44  45 +2            ОТП Банк лицензия № 2766, Москва и обл.   
45  46 +3        ВУЗ-Банк лицензия № 1557, Свердловская обл.   
46  47 +1            МСП Банк лицензия № 3340, Москва и обл.   
47  48 +2            Авангард лицензия № 2879, Москва и обл.   
48  49 −3         Возрождение лицензия № 1439, Москва и обл.   
49  50 +1  Азиатско-Тихоокеанский Банк лицензия № 1810, А...   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [203]:
table.prettify

<bound method Tag.prettify of <table class="standard-table standard-table--row-highlight margin-bottom-small margin-top-x-small" data-test="rating-table">
<thead>
<tr>
<th class="table-title th-sortable th-sortable--asc" rowspan="2">
<a class="th-sortable__text th-sortable__text--double-height" href="/banks/ratings/?sort_param=rating&amp;sort_order=DESC">
                        место                    </a>
</th>
<th class="table-title th-sortable" rowspan="2">
<a class="th-sortable__text th-sortable__text--double-height" href="/banks/ratings/?sort_param=bankname&amp;sort_order=ASC">
                        название банка                        <div class="font-size-small th-sortable__text-sub font-normal">лицензия №, Регион</div>
</a>
</th>
<th class="table-title is-center" colspan="2" width="20%">
                    показатель,
                    тыс. рублей                                                        </th>
<th class="table-title colspan is-center" colspan="2">изменение

# Запрос к API из кода

In [208]:
import requests  
token = '669f554d669f554d669f554d0366e89c596669f669f554d06070c11afa2a9471230bd66'
url = 'https://api.vk.com/method/users.get'   
params = {'user_id': 1, 'v': 5.95, 'fields': 'sex,bdate', 'access_token': token, 'lang': 'ru'}  
  
# Мы можем выставить параметры запроса через аргумент params  
response = requests.get(url, params=params)  
response.text  

'{"response":[{"first_name":"Павел","id":1,"last_name":"Дуров","can_access_closed":true,"is_closed":false,"sex":2,"bdate":"10.10.1984"}]}'

Мы получили строку в формате JSON. Как мы помним по первому разделу, её можно преобразовать в словарь методом json и после этого обращаться к различным полям. Кроме того, такие большие вложенные словари нагляднее выводить с помощью функции pprint (~pretty print, красивый вывод), которым мы и воспользуемся

In [211]:
from pprint import pprint  
  
pprint(response.json())  

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров',
               'sex': 2}]}


In [212]:
user = response.json()['response'][0]  

In [214]:
user['first_name']

'Павел'

Данный метод позволяет запрашивать сразу много пользователей (до 1000).  Для этого нужно использовать параметр user_ids и передавать id через запятую в строковом формате, например: '1,2,3'.

In [216]:
ids = ",".join(map(str, range(1, 4)))  
print(ids)  

1,2,3


In [229]:
params = {'user_ids': ids, 'v': 5.95, 'fields': ['bday','country'], 'access_token': token, 'lang': 'ru'}  

In [230]:
pprint(requests.get(url, params=params).json()) 

{'response': [{'can_access_closed': True,
               'country': {'id': 1, 'title': 'Россия'},
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров'},
              {'can_access_closed': False,
               'first_name': 'Александра',
               'id': 2,
               'is_closed': True,
               'last_name': 'Владимирова'},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 3,
               'last_name': ''}]}


# Задание 1 - Гендерный баланс

Используя API, определите долю женщин(sex=1) среди пользователей с id от 1 до 500. Иногда вам будут попадать пользователи, у которых пол не указан (sex=0), их не нужно учитывать в общем числе.

В ответе укажите число, округлив до двух знаков после запятой, например, 0.55

Пример: если у нас будет 300 пользователей sex=1, 100 пользователей с sex=2 и 100 пользователей с sex=0, то в ответе должно быть 0.75

In [231]:
ids = ','.join(map(str,range(1, 501)))

In [241]:
token = '669f554d669f554d669f554d0366e89c596669f669f554d06070c11afa2a9471230bd66'
url = 'https://api.vk.com/method/users.get'
params = {'user_ids': ids, 'v': 5.95, 'fields': 'sex,bdate',
          'access_token': token, 'lang': 'ru'}

In [253]:
response = requests.get(url, params=params)
response = response.json()

In [262]:
pprint(response)

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров',
               'sex': 2},
              {'can_access_closed': False,
               'first_name': 'Александра',
               'id': 2,
               'is_closed': True,
               'last_name': 'Владимирова',
               'sex': 1},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 3,
               'last_name': '',
               'sex': 0},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 4,
               'last_name': '',
               'sex': 0},
              {'bdate': '18.11',
               'can_access_closed': True,
               'first_name': 'Илья',
               'id': 5,
               'is_closed': False,
               'last_name': 'Перекопский',
       

               'last_name': 'Горбунова',
               'sex': 1},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 125,
               'last_name': '',
               'sex': 0},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 126,
               'last_name': '',
               'sex': 0},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 127,
               'last_name': '',
               'sex': 0},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 128,
               'last_name': '',
               'sex': 0},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 129,
               'last_name': '',
               'sex': 0},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 130,
 

               'can_access_closed': True,
               'first_name': 'Александра',
               'id': 236,
               'is_closed': False,
               'last_name': 'Насекина',
               'sex': 1},
              {'bdate': '15.7.1985',
               'can_access_closed': False,
               'first_name': 'Сергей',
               'id': 237,
               'is_closed': True,
               'last_name': 'Журавлёв',
               'sex': 2},
              {'bdate': '15.8',
               'can_access_closed': False,
               'first_name': 'Татьяна',
               'id': 238,
               'is_closed': True,
               'last_name': 'Тимофеева',
               'sex': 1},
              {'bdate': '26.12',
               'can_access_closed': True,
               'first_name': 'Светлана',
               'id': 239,
               'is_closed': False,
               'last_name': 'Степанова',
               'sex': 1},
              {'deactivated': 'banned',
               'f

               'last_name': '',
               'sex': 0},
              {'bdate': '5.7.1984',
               'can_access_closed': False,
               'first_name': 'Мила',
               'id': 360,
               'is_closed': True,
               'last_name': 'Мамонтова',
               'sex': 1},
              {'bdate': '17.8',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 361,
               'is_closed': False,
               'last_name': 'Кухта',
               'sex': 2},
              {'bdate': '7.8',
               'can_access_closed': True,
               'first_name': 'Елена',
               'id': 362,
               'is_closed': False,
               'last_name': 'Вилинбахова',
               'sex': 1},
              {'bdate': '22.12',
               'can_access_closed': True,
               'first_name': 'Юлия',
               'id': 363,
               'is_closed': False,
               'last_name': 'Кудинова-Ли',
      

               'sex': 1},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 491,
               'last_name': '',
               'sex': 0},
              {'bdate': '12.7.1988',
               'can_access_closed': True,
               'first_name': 'Кирилл',
               'id': 492,
               'is_closed': False,
               'last_name': 'Ширяев',
               'sex': 2},
              {'bdate': '8.5.1985',
               'can_access_closed': True,
               'first_name': 'Дарья',
               'id': 493,
               'is_closed': False,
               'last_name': 'Монихарт',
               'sex': 1},
              {'bdate': '27.12',
               'can_access_closed': True,
               'first_name': 'Анна',
               'id': 494,
               'is_closed': False,
               'last_name': 'Бабурова',
               'sex': 1},
              {'bdate': '24.12.1988',
               'can_access_closed': False,
   

In [270]:
response['response'][500]['sex']

IndexError: list index out of range

In [269]:
len(response['response'])

500

In [284]:
fem_count = 0
lol_count = 0
for user in range(0,len(response['response'])):
    if response['response'][user]['sex'] == 1:
        fem_count += 1
    elif response['response'][user]['sex'] == 0:
        lol_count += 1
round(fem_count/(len(response['response']) - lol_count), 2)

0.48